# KNN近邻分类

### 0.引入库

In [7]:
import numpy as np
import pandas as pd

#引入sklearn里的数据集，iris 鸢尾花
from sklearn.datasets import load_iris

#切分数据集为训练集和测试集的方法
from sklearn.model_selection import train_test_split

#评估函数
from sklearn.metrics import accuracy_score

### 1.数据预处理

In [8]:
iris = load_iris()

df = pd.DataFrame(data = iris.data, columns = iris.feature_names)
df['class'] = iris.target
df['class'] = df['class'].map({i: iris.target_names[i] for i in range(3)})
df.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [9]:
x = iris.data
y = iris.target.reshape(-1, 1)
print(x.shape, y.shape)

(150, 4) (150, 1)


### 2.划分训练集和测试集

In [10]:
#参数设置：test_size是test集占比，random_state随机种子，stratify分层依据，即类别分割比例的标准
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 30, stratify = y)

print(x_train.shape, y_train.shape)

np.sum(np.abs(x_train - x_test[0]), axis = 1)

(105, 4) (105, 1)


array([6.8, 2.2, 2.1, 1.5, 1.3, 3.5, 2. , 4.3, 4.1, 3.6, 3. , 3.6, 1.3,
       0.5, 2.6, 6.6, 6.8, 2.4, 2. , 1. , 6.8, 2.4, 2.9, 2. , 2.2, 6.3,
       2. , 0.5, 6.3, 7.3, 2.2, 7. , 2.5, 2.6, 1.7, 1.7, 7. , 7.7, 7.5,
       1.9, 7.1, 2.5, 1.6, 2.1, 6.7, 1.8, 2.4, 2.3, 7. , 2.9, 7. , 2. ,
       7.2, 1.1, 6.9, 6.7, 2.1, 7.5, 1.8, 6.4, 2.1, 3.7, 1.9, 1.2, 2.6,
       2.2, 7.3, 0.5, 5.2, 4.2, 6.8, 2.3, 2.9, 2.2, 7.2, 2.1, 2. , 1.7,
       8. , 1.8, 2.4, 6.6, 6.6, 2.4, 1.8, 0.7, 4.9, 2. , 2.4, 1.8, 6.8,
       2.9, 7.3, 1.2, 6.9, 2.1, 7.1, 6.7, 2.6, 6.8, 6.8, 7.1, 1.2, 6.5,
       1.9])

### 3.核心算法

In [11]:
#距离函数定义
def l1_distance(a, b):
    return np.sum( np.abs( a - b ), axis = 1) #结果保存为一列的和

def l2_distance(a, b):
    return np.sqrt( np.sum( (a - b) ** 2 , axis = 1))

In [12]:
#分类器实现
class kNN(object):
    #定义一个初始化方法，__init__是类的构造方法
    def __init__(self, n_neighbors = 1, dist_func = l1_distance):
        self.n_neighbors = n_neighbors
        self.dist_func = dist_func
    
    #训练模型方法
    def fit(self, x, y):
        self.x_train = x
        self.y_train = y
        
    #模型预测方法
    def predict(self, x):
        #初始化预测分类数组
        y_prediction = np.zeros( (x.shape[0], 1), dtype = self.y_train.dtype )
        
        #遍历数据集
        for i, x_test in enumerate(x):
            #计算x_test与所有训练数据的距离
            distances = self.dist_func(self.x_train, x_test)
            
            #得到的距离从小到大排序,取出索引
            n_index = np.argsort(distances)
            
            #选取最近的k个点，保存分类类别，ravel()把二维数组展开成一维
            classification = self.y_train[n_index[:self.n_neighbors]].reshape(-1)
            
            #统计出现频率最高的类别，赋值给y_prediction[i]
            y_prediction[i] = np.argmax( np.bincount( classification ) )
        
        return y_prediction

### 4.测试

In [13]:
#定义实例
knn = kNN()

#训练
knn.fit(x_train, y_train)

#保存结果list
result_list = []

#针对不同的参数选取，做预测
for p in [1, 2]:
    knn.dist_func = l1_distance if p == 1 else l2_distance

    for k in range(1, 10, 2):
        knn.n_neighbors = k
        
        #传入测试数据做预测
        prediction = knn.predict(x_test)

        #评估
        accuracy = accuracy_score(y_test, prediction)
        
        result_list.append([k, 'l1_distance' if p == 1 else 'l2_distance', accuracy])

df = pd.DataFrame(result_list, columns = ['k', '距离函数', '预测准确率'])
df

,k,距离函数,预测准确率
0,1,l1_distance,0.933333
1,3,l1_distance,0.933333
2,5,l1_distance,0.911111
3,7,l1_distance,0.911111
4,9,l1_distance,0.933333
5,1,l2_distance,0.933333
6,3,l2_distance,0.933333
7,5,l2_distance,0.933333
8,7,l2_distance,0.933333
9,9,l2_distance,0.933333
